__Программа, поиска числа символов в тексте__

Текст подгружается из файла либо генерируется (на выбор разработчика). Каждый символ кодируется одним байтом. Программа должна корректно работать хотя бы до размера входных данных 4 млн. символов.

## Импорт библиотек

In [5]:
import string  # Библиотека для работы с символьными строками, такими как буквы ASCII
import random  # Библиотека для генерации случайных данных (случайных строк)
from collections import defaultdict  # Библиотека для создания словаря с значениями по умолчанию

import numpy as np
import time

import numba  # Библиотека для ускоренной компиляции функций Python с использованием JIT (Just-In-Time) компиляции
from numba import jit  # Декоратор для указания функции, которую следует ускорить с помощью JIT
from numba import cuda  # Модуль для работы с CUDA (Compute Unified Device Architecture) на GPU с использованием JIT
from numba import prange  # Параллельный range, используется для параллельных циклов с numba

import torch
from torch.nn.parallel import parallel_apply #Паралельные вычисления в PyTorch
import tensorflow as tf


JIT (Just-In-Time) компиляция - это метод оптимизации выполнения программного кода, при котором компиляция выполняется не во время предварительной (статической) компиляции перед запуском программы, а непосредственно во время выполнения программы, перед тем как конкретная часть кода будет выполнена.

Основная идея JIT-компиляции заключается в том, чтобы отложить процесс компиляции до момента, когда он действительно необходим, и при этом скомпилировать только те части кода, которые реально используются в процессе выполнения программы. Это позволяет адаптировать оптимизации к конкретному контексту выполнения, что может привести к улучшению производительности.


prange (параллельный range) в библиотеке Numba представляет собой расширение стандартного цикла range в Python и предназначен для параллельного выполнения итераций цикла. Он позволяет использовать несколько потоков для выполнения итераций цикла одновременно, что может привести к ускорению выполнения в случаях, когда итерации независимы.

## Проверка среды GPU

In [2]:
!nvidia-smi

Tue Dec 12 11:23:43 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
torch.cuda.is_available()
# Output would be True if Pytorch is using GPU otherwise it would be False.

True

In [4]:
tf.test.gpu_device_name()
# Standard output is '/device:GPU:0'

'/device:GPU:0'

## Генерация данных

In [6]:
def generate_random_string(S):
    # join - объединяет этот список символов в строку.
    # random.choices - создает список из S случайно выбранных символов
    # ascii_letters - используем алфавит ASCII (буквы верхнего и нижнего регистра)
    return ''.join(random.choices(string.ascii_letters, k=S))

In [7]:
S = 1_000
text = generate_random_string(S)

print(text)

SoWgxQksOphJcgvjOzwErtglFBWDTPdSeXKNgmnzCnidtYJbINPFFWoSBweFvzZOHNVLrizYfpYISvbOiarnjApkFfIJSUkQhFFSgOHFVgGkzzvjtKnsZQuqMBIjfNDQTJdVsUKZEmKUbSXBFpTMDUlsxmEtNdrZoxhiRgwMfEjSCnVjLNGMWvxmULlGljBOsRoEckwcweNGTTIPqXtkLRanwUeYKarfsDavTzRjrIqKEcObqfQbYwodLaeCueCzqADdBhLZZoRtjfyYAtYryDMQWqKzITJLQZasKTFWQmAeOUwnFFRERgBegqJqYPehZlhhBRwbmxBBgMrADcSoqrEGMHBbNapzRHsyCKsuadeYebFLRHGhLKNUMEKDQyoSXjRpxFTwvLoiukTEnKhrRQlYLbEtuMiuWaGmGbqeMXyGCgaqdkgQbSRRuQEDWFEfJLeHUXWcnXtJwTfhykjNGmXuQhxzHTOXtfOjvGiyzLWHTSDRaycEjqUuzHuLQTpjxirjXzvhHhlnfNRLfQhelLEbPpmHiWyPvphamaLNAtyUFYlMueoNHipxqTzQguKtPXJkgXxoQJCjqhpuMAADLedvGQblzbdjjapriyigFCWOoSdGiEcBdMwREwjPFxxfqglfqlXwSgiIVzYjiaiIYdBBgDVzmvBURbERmAPuiUmuunrIcBIeWMwWwmIbVlQrTjXzILNwlWGUMyOIvWSADkaAKzdkxdqnIGLilcrxjzXmQvbRInkKPRABuLWOhfvLBjZBWcOxeuPPpbZKJykEwYhdEIXEYQjlqLZpdPOFzMROnJHyMLVbLANSqhYKuYCtowgbXVvcoiwNDLarqNEXgCYYYMRLyCpBaZgmwpPTkUXZFisHUcQRCOvbSEBFbYJSqVZPxCLgxfMnyRRZKsEQCDqViHUMUynDiescTBbvVIkFjiCeYCzYhpXRTycFBkNEOXYxLitRrdttsEKIfIbekIoutDcFwZtuMPEoqZGy

In [8]:
S = 40_000_000
text = generate_random_string(S)

## CPU-реализации

### Обычная CPU-реализация

In [18]:
def count_ascii_symbols_cpu(ran):
    # ord(character) возвращает числовое значение ASCII для каждого символа в строке ran
    ASCII_values = [ord(character) for character in ran]
    # Создаем пустой словарь counts для хранения подсчета символов.
    counts = {}
    # Проходимся по каждому числовому значению ASCII в ASCII_values.
    for symbol in ASCII_values:
        if symbol in counts:
            # Если символ уже присутствует в словаре counts, увеличивается его счетчик.
            counts[symbol] += 1
        else:
            # Если символ отсутствует, он добавляется в словарь с начальным счетчиком 1.
            counts[symbol] = 1
    # Возвращаем словарь символов и их частотой встречаемости
    return counts

In [19]:
start_time_CPU = time.time()
counts = count_ascii_symbols_cpu(text)
end_time_CPU = time.time()
all_time_CPU_base = end_time_CPU - start_time_CPU

output_file_path = "output_CPU.txt"

# Открываем файл для записи (режим "w") с использованием указанного пути output_file_path
# Ключевое слово with гарантирует, что файл будет корректно закрыт
# после завершения блока кода, даже если произойдет исключение.
with open(output_file_path, "w") as output_file:
    # Проходимся по каждой паре ключ-значение в словаре counts
    for symbol, count in counts.items():
        # Печатем значения ф-строкой
        output_file.write(f"Символ '{symbol}' повторяется {count} раз(а)\n")

print(f"Результаты сохранены в файл: {output_file_path}")
print(f"Время выполнения (CPU): {all_time_CPU_base:.2f} сек")

Результаты сохранены в файл: output_CPU.txt
Время выполнения (CPU): 8.77 сек


### CPU-реализация с использованием jit-компилятора

In [ ]:
# (nopython=True) - функция полностью компилируется в машинный код
@jit(nopython=True)
def count_ascii_symbols_cpu_jit(ran):
    # ord(character) возвращает числовое значение ASCII для каждого символа в строке ran
    ASCII_values = [ord(character) for character in ran]
    # Создаем пустой словарь counts для хранения подсчета символов.
    counts = {}
    # Проходимся по каждому числовому значению ASCII в ASCII_values.
    for symbol in ASCII_values:
        if symbol in counts:
            # Если символ уже присутствует в словаре counts, увеличивается его счетчик.
            counts[symbol] += 1
        else:
            # Если символ отсутствует, он добавляется в словарь с начальным счетчиком 1.
            counts[symbol] = 1
    # Возвращаем словарь символов и их частотой встречаемости
    return counts

In [ ]:
start_time_CPU = time.time()
counts = count_ascii_symbols_cpu_jit(text)
end_time_CPU = time.time()
all_time_CPU = end_time_CPU - start_time_CPU

output_file_path = "output_CPU_Jit.txt"

with open(output_file_path, "w") as output_file:
    for symbol, count in counts.items():
        output_file.write(f"Символ '{symbol}' повторяется {count} раз(а)\n")

print(f"Результаты сохранены в файл: {output_file_path}")
print(f"Время выполнения (GPU): {all_time_CPU:.2f} сек")

Результаты сохранены в файл: output_CPU_Jit.txt
Время выполнения (GPU): 7.18 сек


### CPU-реализация с паралельными вычислениями

In [ ]:
@jit(nopython=True, parallel=True)
def parallel_count_ascii_symbols_cpu(ran):
    # Инициализация массива счетчиков
    counts = np.zeros(256, dtype=np.int32)
    # Параллельный цикл с использованием prange
    for i in prange(len(ran)):
        symbol = ord(ran[i])
        counts[symbol] += 1  # Увеличение счетчика для соответствующего символа
    return counts

In [ ]:
start_time_СPU_2 = time.time()
counts = parallel_count_ascii_symbols_cpu(text)
end_time_CPU_2 = time.time()
all_time_CPU_2 = end_time_CPU_2 - start_time_СPU_2

output_file_path = "output_CPU_parallel.txt"

with open(output_file_path, "w") as output_file:
    for symbol, count in enumerate(counts):
        if count > 0:
            output_file.write(f"Символ '{chr(symbol)}' повторяется {count} раз(а)\n")

print(f"Результаты сохранены в файл: {output_file_path}")
print(f"Время выполнения (GPU_2): {all_time_CPU_2:.2f} сек")

Результаты сохранены в файл: output_CPU_parallel.txt
Время выполнения (GPU_2): 2.49 сек


### Рализация в TensorFlow

In [ ]:
def count_ascii_symbols_TensorFlow(text):
    # Создаем тензор counts размером 256 элементов, заполненный нулями
    # Вроде как тензер сам выбирает наиболее оптимальное устройство для запуска, в данном случае GPU
    counts = tf.zeros(256, dtype=tf.int64)
    # Преобразование текста в байты (ASCII-коды)
    ascii_values = tf.strings.unicode_decode(tf.strings.regex_replace(text, "[^\x00-\x7F]", ""), 'UTF-8')
    # Используем tf.math.unsorted_segment_sum для подсчета вхождений каждого уникального значения
    counts = tf.tensor_scatter_nd_add(counts, tf.expand_dims(ascii_values, axis=1), tf.ones_like(ascii_values, dtype=tf.int64))
    return counts

In [ ]:
# Замер времени выполнения на GPU
start_time_CPU = time.time()
counts_CPU = count_ascii_symbols_TensorFlow(text)
end_time_CPU = time.time()
all_time_CPU = end_time_CPU - start_time_CPU

# Сохранение результатов в файл
output_file_path_GPU = "output_CPU_TensorFlow.txt"
with open(output_file_path_CPU, "w") as output_file_CPU:
    for symbol, count in enumerate(counts_CPU):
        if count > 0:
            output_file_path_CPU.write(f"Символ '{chr(symbol)}' повторяется {count} раз(а)\n")

print(f"Результаты сохранены в файл: {output_file_path_GPU}")
print(f"Время выполнения (CPU): {all_time_CPU:.2f} сек")

Результаты сохранены в файл: output_CPU_TensorFlow.txt
Время выполнения (GPU): 3.11 сек


### Вторая реализация в TensorFlow


In [ ]:
def count_ascii_symbols_TensorFlow(text):
    # Преобразование текста в байты (ASCII-коды)
    ascii_values = tf.strings.unicode_decode(tf.strings.regex_replace(text, "[^\x00-\x7F]", ""), 'UTF-8')
    # Используем tf.math.bincount для подсчета вхождений каждого уникального значения
    counts = tf.math.bincount(ascii_values, minlength=256, dtype=tf.int64)
    return counts

In [ ]:
# Замер времени выполнения на GPU
start_time_CPU = time.time()
counts_CPU = count_ascii_symbols_TensorFlow(text)
end_time_CPU = time.time()
all_time_CPU = end_time_CPU - start_time_CPU

# Сохранение результатов в файл
output_file_path_GPU = "output_CPU_TensorFlow_2.txt"
with open(output_file_path_GPU, "w") as output_file_GPU:
    for symbol, count in enumerate(counts_CPU):
        if count > 0:
            output_file_GPU.write(f"Символ '{chr(symbol)}' повторяется {count} раз(а)\n")

print(f"Результаты сохранены в файл: {output_file_path_GPU}")
print(f"Время выполнения (GPU): {all_time_CPU:.2f} сек")

Результаты сохранены в файл: output_CPU_TensorFlow_2.txt
Время выполнения (GPU): 3.21 сек


## GPU-реализация

### PyTorch реализация

Здесь текст сначала кодируется в байты с использованием text.encode('ascii'), затем создается тензор ascii_values, содержащий байтовые значения каждого символа. Это решение использует генератор списка для создания списка байтов.

In [ ]:
def count_ascii_symbols_gpu_PyTorch_1(text):
    # Создаём тензор counts размером 256 элементов, заполненный нулями
    # Этот тензор будет использоваться для хранения количества вхождений каждого символа
    # Тип данных установлен как torch.int64 для целых чисел
    # Указано устройство cuda для использования GPU
    counts = torch.zeros(256, dtype=torch.int64, device='cuda')

    # Текст преобразуется в байты (ASCII-коды символов) с использованием text.encode('ascii')
    # создается тензор ascii_values, содержащий байтовые значения каждого символа
    # Тензор затем переносится на устройство cuda для выполнения вычислений на GPU
    ascii_values = torch.ByteTensor([char for char in text.encode('ascii')]).to('cuda')

    # Используется функция torch.bincount для подсчета вхождений каждого уникального значения
    # Результат прибавляется к тензору counts
    # minlength=len(counts) гарантирует, что результат будет иметь тот же размер, что и тензор counts
    counts[:len(counts)] += torch.bincount(ascii_values, minlength=len(counts))

    return counts

In [ ]:
# Замер времени выполнения на GPU
start_time_GPU = time.time()
counts_GPU = count_ascii_symbols_gpu_PyTorch_1(text)
end_time_GPU = time.time()
all_time_GPU = end_time_GPU - start_time_GPU

# Сохранение результатов в файл
output_file_path_GPU = "output_GPU_PyTorch.txt"
with open(output_file_path_GPU, "w") as output_file_GPU:
    for symbol, count in enumerate(counts_GPU):
        if count > 0:
            output_file_GPU.write(f"Символ '{chr(symbol)}' повторяется {count} раз(а)\n")

print(f"Результаты сохранены в файл: {output_file_path_GPU}")
print(f"Время выполнения (GPU): {all_time_GPU:.2f} сек")

Результаты сохранены в файл: output_GPU_PyTorch.txt
Время выполнения (GPU): 4.40 сек


### Вторая PyTorch реализация

Здесь также текст кодируется в байты с использованием text.encode('ascii'), но вместо генератора списка используется bytearray для создания массива байтов, который затем преобразуется в тензор ascii_values.

In [13]:
def count_ascii_symbols_gpu_PyTorch_2(text):
    # Создаём тензор counts размером 256 элементов, заполненный нулями
    # Этот тензор будет использоваться для хранения количества вхождений каждого символа
    # Тип данных установлен как torch.int64 для целых чисел
    # Указано устройство cuda для использования GPU
    counts = torch.zeros(256, dtype=torch.int64, device='cuda')

    # Преобразование текста в байты (ASCII-коды) на GPU
    # Текст преобразуется в байты (ASCII-коды символов) с использованием text.encode('ascii')
    # Создание объекта bytearray из байтов, полученных в результате кодирования текста
    # bytearray представляет собой изменяемую последовательность байтов.
    # torch.ByteTensor(...): Создание PyTorch тензора типа ByteTensor.
    # Этот тип тензора предназначен для хранения байтовых значений.
    # .to('cuda'): Перенос тензора на устройство
    ascii_values = torch.ByteTensor(bytearray(text.encode('ascii'))).to('cuda')

    # Используется функция torch.bincount для подсчета вхождений каждого уникального значения
    # Результат прибавляется к тензору counts
    # minlength=len(counts) гарантирует, что результат будет иметь тот же размер, что и тензор counts

    start_time = time.time()
    counts[:len(counts)] += torch.bincount(ascii_values, minlength=len(counts))
    end_time = time.time()
    all_time = end_time - start_time

    return counts, all_time

In [17]:
# Замер времени выполнения на GPU
start_time_GPU = time.time()
counts_GPU, time_gpu = count_ascii_symbols_gpu_PyTorch_2(text)
end_time_GPU = time.time()
all_time_GPU_best = end_time_GPU - start_time_GPU

# Сохранение результатов в файл
output_file_path_GPU = "output_GPU_PyTorch_2.txt"
with open(output_file_path_GPU, "w") as output_file_GPU:
    for symbol, count in enumerate(counts_GPU):
        if count > 0:
            output_file_GPU.write(f"Символ '{chr(symbol)}' повторяется {count} раз(а)\n")

print(f"Результаты сохранены в файл: {output_file_path_GPU}")
print(f"Общее время выполнения: {all_time_GPU_best:.2f} сек")
print(f"Время вычисления на GPU: {time_gpu:.6f} сек")

Результаты сохранены в файл: output_GPU_PyTorch_2.txt
Общее время выполнения: 4.53 сек
Время вычисления на GPU: 0.000551 сек


## Разница по времени CPU/GPU

In [20]:
all_time_CPU_base / time_gpu

15899.340683095546

## PyTorch на CPU

In [21]:
def count_ascii_symbols_gpu_PyTorch_2(text):
    # Создаём тензор counts размером 256 элементов, заполненный нулями
    # Этот тензор будет использоваться для хранения количества вхождений каждого символа
    # Тип данных установлен как torch.int64 для целых чисел
    # Указано устройство cuda для использования GPU
    counts = torch.zeros(256, dtype=torch.int64)

    # Преобразование текста в байты (ASCII-коды) на GPU
    # Текст преобразуется в байты (ASCII-коды символов) с использованием text.encode('ascii')
    # Создание объекта bytearray из байтов, полученных в результате кодирования текста
    # bytearray представляет собой изменяемую последовательность байтов.
    # torch.ByteTensor(...): Создание PyTorch тензора типа ByteTensor.
    # Этот тип тензора предназначен для хранения байтовых значений.
    # .to('cuda'): Перенос тензора на устройство
    ascii_values = torch.ByteTensor(bytearray(text.encode('ascii')))

    # Используется функция torch.bincount для подсчета вхождений каждого уникального значения
    # Результат прибавляется к тензору counts
    # minlength=len(counts) гарантирует, что результат будет иметь тот же размер, что и тензор counts

    start_time = time.time()
    counts[:len(counts)] += torch.bincount(ascii_values, minlength=len(counts))
    end_time = time.time()
    all_time = end_time - start_time

    return counts, all_time

In [23]:
# Замер времени выполнения на CPU
start_time_GPU = time.time()
counts_GPU, time_gpu = count_ascii_symbols_gpu_PyTorch_2(text)
end_time_GPU = time.time()
all_time_GPU_best = end_time_GPU - start_time_GPU

# Сохранение результатов в файл
output_file_path_GPU = "output_GPU_PyTorch_2.txt"
with open(output_file_path_GPU, "w") as output_file_GPU:
    for symbol, count in enumerate(counts_GPU):
        if count > 0:
            output_file_GPU.write(f"Символ '{chr(symbol)}' повторяется {count} раз(а)\n")

print(f"Результаты сохранены в файл: {output_file_path_GPU}")
print(f"Общее время выполнения: {all_time_GPU_best:.2f} сек")
print(f"Время вычисления на CPU: {time_gpu:.6f} сек")

Результаты сохранены в файл: output_GPU_PyTorch_2.txt
Общее время выполнения: 5.46 сек
Время вычисления на CPU: 0.035332 сек
